In [25]:
import tensorflow as tf
import dataset_helpers as dh
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider
from importlib import reload
reload(dh)

<module 'dataset_helpers' from '/home/DeepMRI/src/dataset_helpers.py'>

In [29]:

train_dataset = dh.load_dataset('../datasets/brats2015-Train-all_training_crop_mri',
                                mri_type=['MR_T1c', 'MR_T2', 'MR_Flair'],
                                random_crop=[160,160,3],
                                batch_size=64,
                                prefetch_buffer=1,
                                shuffle_buffer=128,
                                cache=True,
                                cast_to=np.float32,
                                clip_labels_to=1.0)

In [30]:
def load_all_records(dataset, limit_batches=None):
    iterator = dataset.make_one_shot_iterator()
    next_batch = iterator.get_next()
    batches = []
    with tf.Session() as sess: 
        while True:
            try:
                if len(batches) == (limit_batches or -1):
                    break
                batches += sess.run([next_batch])
            except Exception as e:
                print(e)
                break
    return batches
records = load_all_records(train_dataset, limit_batches=3)

In [31]:
import matplotlib.pyplot as plt
from ipywidgets import IntSlider, interact, fixed
def view_slice(records, r, s):
    batch = records[r]
    mri = batch['mri'][s]
    gt = batch['seg'][s]
    
    plt.figure(figsize=(16,9))
    for i in range(mri.shape[-1]):
        plt.subplot(1, mri.shape[-1]+1, i+1)
        plt.imshow(mri[:,:,i])
    plt.subplot(1, mri.shape[-1]+1, mri.shape[-1]+1)
    plt.imshow(gt[:,:,0])
interact(view_slice, records=fixed(records), r=IntSlider(min=0, max=len(records)-1), s=IntSlider(min=0, max=63))

interactive(children=(IntSlider(value=0, description='r', max=2), IntSlider(value=0, description='s', max=63),…

<function __main__.view_slice(records, r, s)>

In [32]:
records[0]['seg'][0].shape

(160, 160, 1)